## 🔹 Base Model Training — Algorithm Families Overview

Quyida loyihada qo‘llanilgan **asosiy algoritm oilalari** keltirilgan:

- 📏 **Boosting Model**
- 🌳 **Tree-Based Model**

In [22]:
import pandas as pd
import os

In [ ]:
os.chdir(r'C:\SML_Projects\SML_CVE_type_cwe_predict')

In [24]:
x_train = pd.read_csv('data/split/x_train.csv')
x_test = pd.read_csv('data/split/x_test.csv')

y_train = pd.read_csv('data/split/y_train.csv')
y_test = pd.read_csv('data/split/y_test.csv')

In [26]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77655 entries, 0 to 77654
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cve_id        77655 non-null  object
 1   description   77655 non-null  object
 2   cwe           76327 non-null  object
 3   vendor        76530 non-null  object
 4   product       76523 non-null  object
 5   publish_date  77655 non-null  object
dtypes: object(6)
memory usage: 3.6+ MB


In [27]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19414 entries, 0 to 19413
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cve_id        19414 non-null  object
 1   description   19414 non-null  object
 2   cwe           19091 non-null  object
 3   vendor        19151 non-null  object
 4   product       19151 non-null  object
 5   publish_date  19414 non-null  object
dtypes: object(6)
memory usage: 910.2+ KB


In [28]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

In [ ]:
numeric_features = x_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = x_train.select_dtypes(include=['object']).columns

# =======================
# NUMERIC PIPELINE
# =======================
numeric_pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler())
])

# =======================
# CATEGORICAL PIPELINE
# =======================
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(
        handle_unknown='use_encoded_value',
        unknown_value=-1,
        encoded_missing_value=-2
    ))
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_features),
    ('cat', categorical_pipeline, categorical_features)
])

# Logistic Regression pipeline
lr_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('model', LogisticRegression(max_iter=100, random_state=42, n_jobs=-1))
])

# Decision Tree pipeline
dt_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('model', DecisionTreeClassifier(random_state=42))
])

In [30]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score

In [31]:
kf = KFold(n_splits=3, shuffle=True, random_state=42)

### 📌 1. Linear Models 
`LogisticRegression` 

In [32]:
multi_lr = MultiOutputClassifier(lr_pipeline)

multi_lr.fit(x_train, y_train)
lr_y_pred = multi_lr.predict(x_test)

#================[ Target - TYPE ]================
lr_accuracy_type = accuracy_score(y_test['type'], lr_y_pred[:,0])

lr_precision_type = precision_score(y_test['type'], lr_y_pred[:,0], average='macro')
lr_recall_type = recall_score(y_test['type'], lr_y_pred[:,0], average='macro')
lr_f1_type = f1_score(y_test['type'], lr_y_pred[:,0], average='macro')

lr_scores_type = cross_val_score(lr_pipeline, x_train, y_train['type'], cv=kf, scoring='f1_macro')

print(f'Logistic Regression Accuracy for TYPE: {lr_accuracy_type}')

print(f'\nPrecision for TYPE: {lr_precision_type}')
print(f'Recall for TYPE: {lr_recall_type}')
print(f'F1-score for TYPE: {lr_f1_type}')

print("\nK-Fold mean for TYPE:", lr_scores_type.mean())
print("K-Fold std for TYPE:", lr_scores_type.std()) 

print("\nClassification Report for TYPE:\n", classification_report(y_test['type'], lr_y_pred[:,0]))
print('\n========================================================================================')


#================[ Target - CVSS_SCORE ]================
lr_accuracy_cvss_score = accuracy_score(y_test['cvss_score'], lr_y_pred[:,1])

lr_precision_cvss_score = precision_score(y_test['cvss_score'], lr_y_pred[:,1], average='macro')
lr_recall_cvss_score = recall_score(y_test['cvss_score'], lr_y_pred[:,1], average='macro')
lr_f1_cvss_score = f1_score(y_test['cvss_score'], lr_y_pred[:,1], average='macro')

lr_scores_cvss_score = cross_val_score(lr_pipeline, x_train, y_train['cvss_score'], cv=kf, scoring='f1_macro')

print(f'\nLogistic Regression Accuracy for CVSS_SCORE: {lr_accuracy_cvss_score}')

print(f'\nPrecision for CVSS_SCORE: {lr_precision_cvss_score}')
print(f'Recall for CVSS_SCORE: {lr_recall_cvss_score}')
print(f'F1-score for CVSS_SCORE: {lr_f1_cvss_score}')

print("\nK-Fold mean for CVSS_SCORE:", lr_scores_cvss_score.mean())
print("K-Fold std for CVSS_SCORE:", lr_scores_cvss_score.std()) 

print("\nClassification Report for CVSS_SCORE:\n", classification_report(y_test['cvss_score'], lr_y_pred[:,1]))

c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Logistic Regression Accuracy for TYPE: 0.3473781806943443

Precision for TYPE: 0.06895695149334072
Recall for TYPE: 0.122101509619774
F1-score for TYPE: 0.08582927058871112

K-Fold mean for TYPE: 0.0824171855662277
K-Fold std for TYPE: 0.0002641932075326602

Classification Report for TYPE:
                 precision    recall  f1-score   support

    AuthBypass       0.04      0.15      0.06       266
          CSRF       0.00      0.00      0.00       483
           DoS       0.00      0.00      0.00      1104
InfoDisclosure       0.00      0.00      0.00      1892
         Other       0.42      0.55      0.48      7699
 PathTraversal       0.00      0.00      0.00       508
       PrivEsc       0.00      0.00      0.00       357
           RCE       0.00      0.00      0.00      1448
          SQLi       0.00      0.00      0.00      1551
          SSRF       0.00      0.00      0.00       223
           XSS       0.30      0.64      0.41      3883

      accuracy                    

c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag


Logistic Regression Accuracy for CVSS_SCORE: 0.5055114865560936

Precision for CVSS_SCORE: 0.26759025305909456
Recall for CVSS_SCORE: 0.252735954601516
F1-score for CVSS_SCORE: 0.1752360736340849

K-Fold mean for CVSS_SCORE: 0.17163197607156064
K-Fold std for CVSS_SCORE: 0.002920482888519183

Classification Report for CVSS_SCORE:
               precision    recall  f1-score   support

    Critical       0.00      0.00      0.00      1987
        High       0.57      0.02      0.03      6764
         Low       0.00      0.00      0.00       908
      Medium       0.50      0.99      0.67      9755

    accuracy                           0.51     19414
   macro avg       0.27      0.25      0.18     19414
weighted avg       0.45      0.51      0.35     19414



c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag

## 🌲 2. Tree-Based Models

`DecisionTreeRegressor`

In [33]:
multi_dt = MultiOutputClassifier(dt_pipeline)

multi_dt.fit(x_train, y_train)
dt_y_pred = multi_dt.predict(x_test)

#================[ Target - TYPE ]================
dt_accuracy_type = accuracy_score(y_test['type'], dt_y_pred[:,0])

dt_precision_type = precision_score(y_test['type'], dt_y_pred[:,0], average='macro')
dt_recall_type = recall_score(y_test['type'], dt_y_pred[:,0], average='macro')
dt_f1_type = f1_score(y_test['type'], dt_y_pred[:,0], average='macro')

dt_scores_type = cross_val_score(dt_pipeline, x_train, y_train['type'], cv=kf, scoring='f1_macro')

print(f'Logistic Regression Accuracy for TYPE: {dt_accuracy_type}')

print(f'\nPrecision for TYPE: {dt_precision_type}')
print(f'Recall for TYPE: {dt_recall_type}')
print(f'F1-score for TYPE: {dt_f1_type}')

print("\nK-Fold mean for TYPE:", dt_scores_type.mean())
print("K-Fold std for TYPE:", dt_scores_type.std()) 

print("\nClassification Report for TYPE:\n", classification_report(y_test['type'], dt_y_pred[:,0]))
print('\n========================================================================================')


#================[ Target - CVSS_SCORE ]================
dt_accuracy_cvss_score = accuracy_score(y_test['cvss_score'], dt_y_pred[:,1])

dt_precision_cvss_score = precision_score(y_test['cvss_score'], dt_y_pred[:,1], average='macro')
dt_recall_cvss_score = recall_score(y_test['cvss_score'], dt_y_pred[:,1], average='macro')
dt_f1_cvss_score = f1_score(y_test['cvss_score'], dt_y_pred[:,1], average='macro')

dt_scores_cvss_score = cross_val_score(dt_pipeline, x_train, y_train['cvss_score'], cv=kf, scoring='f1_macro')

print(f'\nDecisionTree Accuracy for CVSS_SCORE: {dt_accuracy_cvss_score}')

print(f'\nPrecision for CVSS_SCORE: {dt_precision_cvss_score}')
print(f'Recall for CVSS_SCORE: {dt_recall_cvss_score}')
print(f'F1-score for CVSS_SCORE: {dt_f1_cvss_score}')

print("\nK-Fold mean for CVSS_SCORE:", dt_scores_cvss_score.mean())
print("K-Fold std for CVSS_SCORE:", dt_scores_cvss_score.std()) 

print("\nClassification Report for CVSS_SCORE:\n", classification_report(y_test['cvss_score'], dt_y_pred[:,1]))

Logistic Regression Accuracy for TYPE: 0.47646028639126403

Precision for TYPE: 0.5905354286747948
Recall for TYPE: 0.5902474274722217
F1-score for TYPE: 0.563772929603936

K-Fold mean for TYPE: 0.5776033642991631
K-Fold std for TYPE: 0.03169590029075161

Classification Report for TYPE:
                 precision    recall  f1-score   support

    AuthBypass       0.77      0.70      0.74       266
          CSRF       0.72      0.60      0.65       483
           DoS       0.15      0.55      0.24      1104
InfoDisclosure       0.56      0.53      0.54      1892
         Other       0.50      0.36      0.42      7699
 PathTraversal       0.51      0.71      0.59       508
       PrivEsc       0.23      0.48      0.31       357
           RCE       0.20      0.36      0.26      1448
          SQLi       0.99      0.89      0.94      1551
          SSRF       0.95      0.86      0.90       223
           XSS       0.91      0.46      0.61      3883

      accuracy                       

In [34]:
results = x_test[['description', 'product', 'publish_date']].copy()

results['type'] = y_test['type'].values
results['cvss_score'] = y_test['cvss_score'].values

results['lr_pred_type'] = lr_y_pred[:, 0]
results['lr_pred_cvss'] = lr_y_pred[:, 1]

results['dt_pred_type'] = dt_y_pred[:, 0]
results['dt_pred_cvss'] = dt_y_pred[:, 1]

results.sort_values(by=['type', 'cvss_score'], ascending=False).head(20)

,description,product,publish_date,type,cvss_score,lr_pred_type,lr_pred_cvss,dt_pred_type,dt_pred_cvss
11,Improper Neutralization of Input During Web Pa...,wp,2025-04-16,XSS,Medium,Other,Medium,Other,Medium
26,Improper Neutralization of Input During Web Pa...,coyier,2025-06-20,XSS,Medium,XSS,Medium,Other,Critical
36,Improper Neutralization of Input During Web Pa...,utech,2024-12-16,XSS,Medium,XSS,Medium,XSS,Medium
39,The ACF Onyx Poll plugin for WordPress is vuln...,wordpress,2025-06-13,XSS,Medium,XSS,Medium,Other,Medium
47,Adobe Experience Manager versions 6.5.18 and e...,experience_manager,2023-12-15,XSS,Medium,XSS,Medium,Other,High
49,Improper Neutralization of Input During Web Pa...,pacsrapor,2023-03-21,XSS,Medium,XSS,Medium,XSS,Critical
65,The web interface of Gira Giersiepen Gira KNX/...,knx_ip_router_firmware,2023-06-30,XSS,Medium,XSS,Medium,XSS,Critical
75,WeGIA is a Web manager for charitable institut...,wegia,2025-03-27,XSS,Medium,Other,Medium,XSS,Medium
76,The Login Configurator WordPress plugin throug...,login_configurator,2023-07-17,XSS,Medium,XSS,Medium,XSS,Critical
98,A cross site scripting vulnerability in the AR...,webserv1,2024-01-31,XSS,Medium,XSS,Medium,Other,Low


In [35]:
from rich.table import Table
from rich.console import Console

console = Console(width=160)


results = [
    # Linear Family
    [
        'LogisticRegression', 

        lr_accuracy_type, 
        lr_precision_type,
        lr_recall_type,
        lr_f1_type,
        lr_scores_type.mean(), 
        lr_scores_type.std(), 

        lr_accuracy_cvss_score, 
        lr_precision_cvss_score,
        lr_recall_cvss_score,
        lr_f1_cvss_score,
        lr_scores_cvss_score.mean(), 
        lr_scores_cvss_score.std()
    ],

    # Tree-Based Family
    [
        'DecisionTree', 

        dt_accuracy_type, 
        dt_precision_type,
        dt_recall_type,
        dt_f1_type,
        dt_scores_type.mean(), 
        dt_scores_type.std(), 

        dt_accuracy_cvss_score, 
        dt_precision_cvss_score,
        dt_recall_cvss_score,
        dt_f1_cvss_score,
        dt_scores_cvss_score.mean(), 
        dt_scores_cvss_score.std()
    ],
    
]

for row in results:
    type_acc = row[1]
    cvss_score_acc = row[7]
    combined = (type_acc + cvss_score_acc) / 2
    row.append(combined)

result_sorted = sorted(results, key=lambda i: i[3] and i[9], reverse=True)
best_model = max(results, key=lambda x: x[3] and x[9])

table = Table(title="Base model training Comparison", show_lines=True)
table.add_column("Model", justify="center", vertical="middle")
table.add_column("TYPE Accuracy", justify="center", vertical="middle")
table.add_column("TYPE Precision", justify="center", vertical="middle")
table.add_column("TYPE Recall", justify="center", vertical="middle")
table.add_column("TYPE F1-score", justify="center", vertical="middle")
table.add_column("TYPE CV mean", justify="center", vertical="middle")
table.add_column("TYPE CV std", justify="center", vertical="middle")

table.add_column("CVSS Accuracy", justify="center", vertical="middle")
table.add_column("CVSS Precision", justify="center", vertical="middle")
table.add_column("CVSS Recall", justify="center", vertical="middle")
table.add_column("CVSS F1-score", justify="center", vertical="middle")
table.add_column("CVSS CV mean", justify="center", vertical="middle")
table.add_column("CVSS CV std", justify="center", vertical="middle")

table.add_column("Combined", justify="center", vertical="middle")


for row in result_sorted:
    algo, type_acc, type_pre, type_recall, type_f1, type_kmean, type_kstd, cvss_score_acc, cvss_score_pre, cvss_score_recall, cvss_score_f1, cvss_score_kmean, cvss_score_kstd, combined = row

    if row == best_model:
        table.add_row(
            # TYPE
            f"[bold green]{algo}[/bold green]",
            f"[bold green]{type_acc:.2f}[/bold green]",
            f"[bold green]{type_pre:.2f}[/bold green]",
            f"[bold green]{type_recall:.2f}[/bold green]",
            f"[bold green]{type_f1:.2f}[/bold green]",
            f"[bold green]{type_kmean:.2f}[/bold green]",
            f"[bold green]{type_kstd:.2f}[/bold green]",

            # CVSS_SCORE
            f"[bold green]{cvss_score_acc:.2f}[/bold green]",
            f"[bold green]{cvss_score_pre:.2f}[/bold green]",
            f"[bold green]{cvss_score_recall:.2f}[/bold green]",
            f"[bold green]{cvss_score_f1:.2f}[/bold green]",
            f"[bold green]{cvss_score_kmean:.2f}[/bold green]",
            f"[bold green]{cvss_score_kstd:.2f}[/bold green]",
            f"[bold green]{combined:.2f}[/bold green]",
        )
    else: table.add_row(algo, f"{type_acc:.2f}", f"{type_pre:.2f}", f"{type_recall:.2f}", f"{type_f1:.2f}", f"{type_kmean:.2f}", f"{type_kstd:.2f}",
                      f"{cvss_score_acc:.2f}", f"{cvss_score_pre:.2f}", f"{cvss_score_recall:.2f}", f"{cvss_score_f1:.2f}", f"{cvss_score_kmean:.2f}", f"{cvss_score_kstd:.2f}", f"{combined:.2f}")

console.print(table)

                                                                 Base model training Comparison                                                                 
┏━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓
┃          ┃   TYPE   ┃   TYPE   ┃   TYPE    ┃   TYPE   ┃  TYPE CV  ┃ TYPE CV  ┃   CVSS    ┃   CVSS   ┃   CVSS    ┃   CVSS   ┃  CVSS CV  ┃ CVSS CV  ┃          ┃
┃  Model   ┃ Accuracy ┃ Precisi… ┃  Recall   ┃ F1-score ┃   mean    ┃   std    ┃ Accuracy  ┃ Precisi… ┃  Recall   ┃ F1-score ┃   mean    ┃   std    ┃ Combined ┃
┡━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩
│ Decisio… │   0.48   │   0.59   │   0.59    │   0.56   │   0.58    │   0.03   │   0.39    │   0.28   │   0.27    │   0.27   │   0.28    │   0.01   │   0.43   │
├──────────┼──────────┼──────────┼───────────┼──────────┼───────────┼──────────┼───────────┼──────────┼───────────┼──────────┼───────────┼──────────┼──────────┤
│ Logisti… │   0.35   │   0.07   │   0.12    │   0.09   │   0.08    │   0.00   │   0.51    │   0.27   │   0.25    │   0.18   │   0.17    │   0.00   │   0.43   │
└──────────┴──────────┴──────────┴───────────┴──────────┴───────────┴──────────┴───────────┴──────────┴───────────┴──────────┴───────────┴──────────┴──────────┘

In [37]:
os.makedirs('results', exist_ok=True)

temp_console = Console(record=True, width=160)
temp_console.print(table)
text = temp_console.export_text()
with open('results/baseline_model.txt', 'w', encoding='utf-8') as f:
    f.write(text)

                                                                 Base model training Comparison                                                                 
┏━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓
┃          ┃   TYPE   ┃   TYPE   ┃   TYPE    ┃   TYPE   ┃  TYPE CV  ┃ TYPE CV  ┃   CVSS    ┃   CVSS   ┃   CVSS    ┃   CVSS   ┃  CVSS CV  ┃ CVSS CV  ┃          ┃
┃  Model   ┃ Accuracy ┃ Precisi… ┃  Recall   ┃ F1-score ┃   mean    ┃   std    ┃ Accuracy  ┃ Precisi… ┃  Recall   ┃ F1-score ┃   mean    ┃   std    ┃ Combined ┃
┡━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩
│ Decisio… │   0.48   │   0.59   │   0.59    │   0.56   │   0.58    │   0.03   │   0.39    │   0.28   │   0.27    │   0.27   │   0.28    │   0.01   │   0.43   │
├──────────┼──────────┼──────────┼───────────┼──────────┼───────────┼──────────┼───────────┼──────────┼───────────┼──────────┼───────────┼──────────┼──────────┤
│ Logisti… │   0.35   │   0.07   │   0.12    │   0.09   │   0.08    │   0.00   │   0.51    │   0.27   │   0.25    │   0.18   │   0.17    │   0.00   │   0.43   │
└──────────┴──────────┴──────────┴───────────┴──────────┴───────────┴──────────┴───────────┴──────────┴───────────┴──────────┴───────────┴──────────┴──────────┘

# Next -->

<a href="offline_testing.ipynb" target="_blank" style="text-decoration: none;">
    <button style="
        background-color: #45a049;
        color: white; 
        padding: 12px 24px; 
        font-size: 16px; 
        border: none; 
        border-radius: 8px;
        cursor: pointer;
        transition: background-color 0.3s;
    " onmouseover="this.style.backgroundColor='#45a049'" onmouseout="this.style.backgroundColor='#4CAF50'">
        Offline Testing
    </button>
</a>

<a href="Analysis/EDA.ipynb" target="_blank" style="text-decoration: none;">
    <button style="
        background-color: #45a049;
        color: white; 
        padding: 12px 24px; 
        font-size: 16px; 
        border: none; 
        border-radius: 8px;
        cursor: pointer;
        transition: background-color 0.3s;
    " onmouseover="this.style.backgroundColor='#45a049'" onmouseout="this.style.backgroundColor='#4CAF50'">
        EDA
    </button>
</a>

<a href="Analysis/SHAP.ipynb" target="_blank" style="text-decoration: none;">
    <button style="
        background-color: #45a049;
        color: white; 
        padding: 12px 24px; 
        font-size: 16px; 
        border: none; 
        border-radius: 8px;
        cursor: pointer;
        transition: background-color 0.3s;
    " onmouseover="this.style.backgroundColor='#45a049'" onmouseout="this.style.backgroundColor='#4CAF50'">
        SHAP
    </button>
</a>